In [1]:
from data.Safe2Unsafe import DeepAccidentDataset
from method.dynamics import HyperplaneEncoder
from method.barriers import DiscriminatingHyperplane
from method.trainers import HyperplaneTrainer
import torch
import time
import pytorch_lightning as pl
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.val_dataloader()

In [10]:
latent_dim = 16
barrier = DiscriminatingHyperplane(2,latent_dim=latent_dim)
# model = HyperplaneEncoder(2,"cuda",model="google/vit-base-patch16-224",latent_dim=latent_dim)
model = HyperplaneEncoder(2,"cuda",model="openai/clip-vit-base-patch16",latent_dim=latent_dim)
# model = HyperplaneEncoder(2,"cuda",model="resnet",latent_dim=latent_dim)
trainer = HyperplaneTrainer(model,barrier)
checkpoint = torch.load("/root/tf-logs/DiscriminatingHyperplane/version_1/checkpoints/last.ckpt")
trainer.load_state_dict(checkpoint['state_dict'])
trainer.to("cuda")

HyperplaneTrainer(
  (model): HyperplaneEncoder(
    (encoder): ViTAttentionEncoder(
      (ViT): CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
          (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
            (position_embedding): Embedding(197, 768)
          )
          (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder): CLIPEncoder(
            (layers): ModuleList(
              (0): CLIPEncoderLayer(
                (self_attn): CLIPAttention(
                  (k_proj): Linear(in_features=768, out_features=768, bias=True)
                  (v_proj): Linear(in_features=768, out_features=768, bias=True)
                  (q_proj): Linear(in_features=768, out_features=768, bias=True)
                  (out_proj): Linear(in_features=768, out_features=768, bias=True)
                )
                (layer_norm1): LayerNorm((768,), eps=1e-

In [11]:
b_all = []
label_all = []
trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x = trainer.model.simulate(i,u)
    a,b = trainer.barrier(x)
    value = torch.einsum("btc,btc->bt",a,u) + b
    b_all.append((value < 0).cpu())
    label_all.append(label.squeeze(-1))
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_hyperplane_clip.txt",results)

100%|██████████| 4/4 [00:00<00:00,  4.43it/s]


In [12]:
results = np.loadtxt("./results_hyperplane_clip.txt")
regular = results[results[:,-1] == 0,:-1]
collision = results[results[:,-1] == 1,:-1]
acc_regular = (regular <= 0).mean()
acc_collision = (collision > 0).mean()
print(acc_regular,acc_collision)

0.4227814569536424 0.17333333333333334


In [7]:
regular

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])